In [1]:
Category = {
    0:"Samdasoo_pet",
    1:"CornsilkTea_pet",
    2:"Cocacola_can",
    3:"Pepsi_can",
    4:"BananaMilk",
    5:"Coolpeace_carton",
    6:"LetsbeMild_can",
    7:"MycafelatteMild",
    8:"MinutemaidOrange_pet",
    9:"BacchusF_glass",
}


In [1]:
# test dataset img 저장
from matplotlib import pyplot as plt
from os import listdir
from os.path import isfile, join
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data.datasets.coco import load_coco_json
import os

model_name = 'faster_rcnn_R_101_C4_3x'

cfg = get_cfg()
cfg.OUTPUT_DIR = './logs/' + model_name
cfg.merge_from_file(cfg.OUTPUT_DIR + '/config.yaml')
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_0089999.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10
predictor = DefaultPredictor(cfg)

AssertionError: Config file './logs/faster_rcnn_R_101_C4_3x/config.yaml' does not exist!

In [3]:
def get_NormalBox(output):
    """
    protobuf 통신을 위한 Detectron2 모델의 inference 결과를 반환하는 함수
    
    Arg:
        output (Boxes class): model이 예측한 결과물
    
    Return:
    
    """
    t_pred_classes = output['instances'].pred_classes
    classes = t_pred_classes.to("cpu").numpy()
    
    t_score = output['instances'].scores
    scores = t_score.to("cpu").numpy()

    t_boxes = output["instances"].pred_boxes
    bboxes = t_boxes.tensor.to("cpu").numpy()
    
    for idx in classes:
        res = get_info(bboxes[idx], classes[idx], scores[idx])
        print(res)

def get_info(bbox, class_id, score):
    """
    Args를 취합하여 protobuf에 정의된 형식에 맞게 반환
    
    Args:
        bbox (1D-list): bbox 정보
        class_id (int): class id
        score (int): confidence score
    
    Return:
        1D-list: [xmin, ymin, xmax, ymax, class_info, score]
        xmin (int32): bbox의 min x 좌표
        ymin (int32): bbox의 min y 좌표
        xmax (int32): bbox의 max x 좌표
        ymax (int32): bbox의 max y 좌표
        class_info (string): 숫자로 표현된 class ->
                    Category에 정의된 dictionary를 이용해 string class로 바꿔줌
        score (float): confidence score
    """
    xmin = int(bbox[0])
    ymin = int(bbox[1])
    xmax = int(bbox[2])
    ymax = int(bbox[3])
    class_info = Category[class_id]
   
    return [xmin, ymin, xmax, ymax, class_info, score]

In [4]:
import os
import cv2
import random
from matplotlib import pyplot as plt

root_dir = "/detectron2_repo/datasets/data_voucher_2019"
group_dir = join(root_dir, "group1")
json_file = join(root_dir, group_dir, "annotations_json/instances_val.json")
image_root = join(root_dir, group_dir, "test_images")

img_list = [join(image_root, f) for f in sorted(listdir(image_root))]
print(img_list)

['/detectron2_repo/datasets/data_voucher_2019/group1/test_images/003431.jpg', '/detectron2_repo/datasets/data_voucher_2019/group1/test_images/003432.jpg', '/detectron2_repo/datasets/data_voucher_2019/group1/test_images/003433.jpg', '/detectron2_repo/datasets/data_voucher_2019/group1/test_images/003434.jpg', '/detectron2_repo/datasets/data_voucher_2019/group1/test_images/003435.jpg', '/detectron2_repo/datasets/data_voucher_2019/group1/test_images/003436.jpg', '/detectron2_repo/datasets/data_voucher_2019/group1/test_images/003437.jpg', '/detectron2_repo/datasets/data_voucher_2019/group1/test_images/003438.jpg', '/detectron2_repo/datasets/data_voucher_2019/group1/test_images/003439.jpg', '/detectron2_repo/datasets/data_voucher_2019/group1/test_images/003440.jpg', '/detectron2_repo/datasets/data_voucher_2019/group1/test_images/003441.jpg', '/detectron2_repo/datasets/data_voucher_2019/group1/test_images/003442.jpg', '/detectron2_repo/datasets/data_voucher_2019/group1/test_images/003443.jpg'

In [7]:
for img_dir in random.sample(img_list, 1):    
    im = cv2.imread(img_dir)
    im = cv2.resize(im, (660, 585))
    output = predictor(im)
  
    get_NormalBox(output)

AttributeError: 'dict' object has no attribute 'inference'

In [ ]:
# 요청한 함수
product_list = ["Samdasoo_pet",
                "CornsilkTea_pet",
                "Cocacola_can",]


def softmax_func(output, product_list):
    """
    Detectron2가 예측한 전체 클래스별 확률값을
    product_list에 있는 클래스별 확률값으로 변환시킴
    
    Args:
        output (dict): Detectron2가 예측한 전체 클래스별 확률값
        product_list (list): 물품의 리스트 
    """
    print(output['instances'])

prediction_result = softmax_func(output, product_list)

In [ ]:
# 결과 확인
t_pred_classes = output['instances'].pred_classes
classes = t_pred_classes.to("cpu").numpy()
    
t_score = output['instances'].scores
scores = t_score.to("cpu").numpy()

t_boxes = output["instances"].pred_boxes
bboxes = t_boxes.tensor.to("cpu").numpy()

im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

for idx in range(len(classes)):
    res = get_info(bboxes[idx], classes[idx], scores[idx])
    print(res)
    im = cv2.rectangle(im, (res[0], res[1]), (res[2], res[3]), color=(0, 0, 0))
    
plt.imshow(im)

In [ ]:
print(type(output))

In [ ]:
output.keys()

In [ ]:
instances = output['instances']
print(instances)

In [ ]:
print(*instances.pred_boxes[0].tensor.to('cpu').numpy()[0])

In [ ]:
len(instances)